In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_lg
# !pip install bert_score

# Load Libraries

In [1]:
from functions import *
from Sentence_ import Sentence_
from Ticket_ import Ticket_
from Ticket_Pair_ import Ticket_Pair_
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cloud_user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from bert_score import score

In [4]:
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/cloud_user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load Dataset

In [5]:
final_to_consider = pd.read_pickle('"-- Set Respective Path HERE --" ')

# Split Train/Test Data

In [6]:
sorted_idx = final_to_consider.ticket_obj.map(lambda x: x.date).sort_values().index
final_to_consider = final_to_consider.loc[sorted_idx]

train_percentage = 0.80
test_percentage = 0.20
train_break = int(final_to_consider.shape[0]*train_percentage)

train_df = final_to_consider[:train_break]
test_df = final_to_consider[train_break:]

In [7]:
train_df.shape, test_df.shape

((13600, 3), (3400, 3))

In [ ]:
# train_df = final_to_consider[final_to_consider.is_train==True]
# test_df = final_to_consider[final_to_consider.is_train==False]

# Bag of Words Using TF-IDF

In [8]:

problem_vectorizer=TfidfVectorizer(use_idf=True, stop_words = stopWords)
problem_vectorizer.fit(train_df.ticket_obj.map(lambda x: x.content.text).values.tolist())

content_train_vecs = problem_vectorizer.transform(train_df.ticket_obj.map(lambda x: x.content.text).values.tolist()).toarray()
content_test_vecs = problem_vectorizer.transform(test_df.ticket_obj.map(lambda x: x.content.text).values.tolist()).toarray()


In [9]:
res_vectorizer = TfidfVectorizer(use_idf=True, stop_words = stopWords)
res_corpus = list(itertools.chain(*train_df.res_actions.values.tolist()))
res_vectorizer.fit(res_corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

# Apply KNN using TF-IDF vectors

In [10]:
def get_neighbors_from_emb(test_vecs, train_vecs, k=5, threshold=0.7):
    
    if (len(test_vecs)==0) or (len(train_vecs)==0):
        return pd.DataFrame(columns=['neighbors'])
    
    test_emb = np.matrix(test_vecs.tolist())
    train_emb = np.matrix(train_vecs.tolist())
    
    similarity = cosine_similarity(test_emb, train_emb)
    neighbors = []
    similarity = np.round(similarity, 2)
    
    for i in range(similarity.shape[0]): 
        states = (similarity[i] >= threshold) & (similarity[0] <= 1)
        indices = np.where(states)[0]
        indices = indices[np.argsort(similarity[i][indices])[::-1][:k]]
        sim_scores = np.round(similarity[i][indices], 4)
        neighbors.append(list(tuple(zip(indices, sim_scores))))
    
    return pd.DataFrame([neighbors], index=['neighbors']).T

In [11]:
def map_best_actions(actual_actions, predicted_actions, vectorizer_):
    
    if (len(actual_actions)==0) or (len(predicted_actions)==0):
        return pd.DataFrame()
    
    actual_actions_vecs = vectorizer_.transform(actual_actions).toarray()
    predicted_actions_vecs = vectorizer_.transform(predicted_actions).toarray()
    mapping_ = get_neighbors_from_emb(actual_actions_vecs, predicted_actions_vecs, k=1, threshold=0.0)
    mapping_.index = actual_actions
    mapping_ = mapping_[mapping_.neighbors.map(len)>0]
    try:
        mapping_.neighbors = mapping_.neighbors.map(lambda x: (x[0][0], predicted_actions[x[0][0]], x[0][1]))
    except:
#         print (mapping_)
        raise Exception
    
    return mapping_

In [12]:
test_neighbors = get_neighbors_from_emb(content_test_vecs, content_train_vecs,k=10)

In [13]:
""" Map Index of Neighbor """
test_neighbors.neighbors = test_neighbors.neighbors.map(lambda x: [(train_df.iloc[[a[0]]].index[0], a[1]) for a in x])
test_df.loc[:,'all_neighbors'] = test_neighbors.neighbors.values.tolist()

/home/cloud_user/anaconda3/envs/ticket_resolution/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/cloud_user/anaconda3/envs/ticket_resolution/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# Evaluation of Results

In [14]:
compute_meteor_score = lambda actual, prediction: round(meteor_score([actual], prediction), 4)
compute_bert_scores = lambda actual_list, pred_list : np.round((score(pred_list, actual_list, lang='en', model_type='bert-base-uncased', nthreads=32,rescale_with_baseline=True)[-1]).tolist(),4)



In [15]:
def get_mapped_data(result_df, inc='None'):
    
    if result_df.shape[0] < 1:
        return []
    
    incs = [inc]*result_df.shape[0]
    actual_labels = result_df.index.tolist()
    predicted_labels = result_df.neighbors.map(lambda x: x[1])
    return list(zip(incs, actual_labels, predicted_labels))

In [16]:
def generate_best_meteor_scores(df_):
    """ Input : df_['best_actions'] """
    
    df_.loc[:,'best_meteor_scores'] = df_.mapped_actions.apply(lambda r: [compute_meteor_score(res[1],res[2]) for res in get_mapped_data(r)])
    return df_

In [17]:
def generate_best_bert_scores(df_):
    """ Input : df_['best_actions'] """
    
    bert_score_data = df_.apply(lambda x: get_mapped_data(x['mapped_actions'], x.ticket_obj.number) , axis=1)
    bert_score_data = pd.DataFrame(list(itertools.chain(*bert_score_data.values.tolist())), columns=['number','actual','predicted'])
    bert_score_data.loc[:,'bert_score'] = compute_bert_scores(bert_score_data['actual'].values.tolist(), bert_score_data['predicted'].values.tolist())
    bert_score_data.loc[bert_score_data.bert_score<0, 'bert_score'] = 0.0
    bert_score_data.index = bert_score_data.number.values.tolist()
    df_.loc[:,'best_bert_scores'] = df_.ticket_obj.map(lambda x: bert_score_data.bert_score.loc[[x.number]].values.tolist() if x.number in bert_score_data.index else [])
    return df_
    



In [19]:
def evaluate_on_k(test_df, k=5):

    test_df.loc[:,'neighbors'] = test_df.all_neighbors.map(lambda x: x[:k])
    test_df.loc[:,'predicted_actions'] = test_df.neighbors.apply(lambda x: set(itertools.chain(*[train_df.res_actions.loc[a[0]] for a in x]))).map(list)
    test_df.loc[:,'mapped_actions'] = test_df.apply(lambda x: map_best_actions(x['res_actions'],x['predicted_actions'], res_vectorizer), axis=1)
    test_df = generate_best_meteor_scores(test_df)
    test_df = generate_best_bert_scores(test_df)
    test_df.loc[:,'scores_sum'] = test_df.apply(lambda x: (np.mean([x['best_meteor_scores'], x['best_bert_scores']], axis=0).sum()), axis=1) 
    test_df.loc[:, 'recall'] = test_df.apply(lambda x: x['scores_sum']/len(x['res_actions']), axis=1).values.tolist()
    test_df.loc[:, 'precision'] = test_df.apply(lambda x: x['scores_sum']/len(x['predicted_actions']) if len(x['predicted_actions'])!=0 else 0, axis=1).values.tolist()
    test_df.loc[:, 'precision'] = test_df.precision.map(lambda x: 1.0 if x > 1.0 else x)
    test_df.loc[:, 'recall'] = test_df.recall.map(lambda x: 1.0 if x > 1.0 else x)
    beta = 1
    test_df.loc[:, 'f1_score'] = (((1+beta**2)*test_df['precision']*test_df['recall'])/((beta**2)*test_df['precision']+test_df['recall'])).fillna(0).values.tolist()


    return test_df[['precision','recall','f1_score']].mean()

In [20]:
results = pd.DataFrame(columns=['K','Average Precision','Average Recall','Average F1-Score'])
for k in range(1,11):
    r = evaluate_on_k(test_df, k=k)
    results.loc[k] = [k]+r.values.round(4).tolist()

/home/cloud_user/anaconda3/envs/ticket_resolution/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/cloud_user/anaconda3/envs/ticket_resolution/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/cloud_user/anaconda3/envs/ticket_resolution/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is try

In [21]:
results

,K,Average Precision,Average Recall,Average F1-Score
1,1.0,0.4350,0.4366,0.4259
2,2.0,0.3996,0.4852,0.4189
3,3.0,0.3828,0.5014,0.4083
4,4.0,0.3739,0.5111,0.4015
5,5.0,0.3690,0.5133,0.3965
6,6.0,0.3615,0.5179,0.3909
7,7.0,0.3597,0.5198,0.3889
8,8.0,0.3563,0.5243,0.3857
9,9.0,0.3538,0.5257,0.3825
10,10.0,0.3528,0.5265,0.3811


In [22]:
results.to_pickle('../data/Bow-tdidf_results.pkl')